In [187]:
from IPython.display import IFrame
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as patches
import warnings
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import zipfile

from IPython.display import IFrame
from shapely.geometry import Point, LineString, shape, Polygon


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Testing Out GTFS Data Visualization using NYC Ferry GTFS
## NYC Ferry GTFS Contains:
- agency.txt
- routes.txt
- shapes.txt
- stops.txt
- trips.txt

With the above files from any GTFS data, I should be able to produce a quick transit map of any city which produces GTFS data.
Reding in all data here:

In [228]:
city_name = 'fairfax_cue'
z = zipfile.ZipFile(f"{city_name}_gtfs.zip")
z.extractall(f"{city_name}_gtfs")

In [229]:
#Read in all necessary data from GTFS folder
agency = pd.read_csv(f"{city_name}_gtfs/agency.txt")
routes = pd.read_csv(f"{city_name}_gtfs/routes.txt")
shapes = pd.read_csv(f"{city_name}_gtfs/shapes.txt")
stops = pd.read_csv(f"{city_name}_gtfs/stops.txt")
trips = pd.read_csv(f"{city_name}_gtfs/trips.txt")

In [230]:
# Taking agency name to save html file at the end.
html_name = agency['agency_name'][0]

In [231]:
html_name

'Fairfax CUE'




## Mapping the Stops
GTFS Data contains coordinates for stops along all the lines.

In [232]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   route_id       592 non-null    object
 1   service_id     592 non-null    object
 2   trip_id        592 non-null    int64 
 3   trip_headsign  592 non-null    object
 4   block_id       592 non-null    int64 
 5   shape_id       592 non-null    object
dtypes: int64(2), object(4)
memory usage: 27.9+ KB


In [233]:
#read in stops as gdf
stops_gdf = gpd.GeoDataFrame(
    stops,
    crs=4326,
    geometry = gpd.points_from_xy(stops.stop_lon, stops.stop_lat))




## Mapping the Routes

In order to map out the routes in a line, first I will need to convert points from shape.txt, merge routes.txt for route name and other info.

In [234]:
# Create GeoDataFrame from Shapes.txt
shapes_gdf = gpd.GeoDataFrame(
    shapes,
    crs=4326,
    geometry = gpd.points_from_xy(shapes.shape_pt_lon, shapes.shape_pt_lat)
)

In [235]:
# Create Linestring out of shape_id and shape_pt_sequence
shapes_gdf = shapes_gdf.groupby(['shape_id'])['geometry'].apply(lambda x: LineString(x.tolist()))
shapes_gdf = gpd.GeoDataFrame(shapes_gdf, geometry='geometry')

In [236]:
#keeping trips info only for each trips to take out trip_id
trips_drop = trips.drop_duplicates(subset=['shape_id'])

In [237]:
#merging trip_id on to shapes in order to merge into route.txt
shape_merge = shapes_gdf.merge(trips_drop,left_on='shape_id', right_on='shape_id', how='outer')

In [238]:
#dropping duplicate shapes to simplify route for this test
shape_merge = shape_merge.drop_duplicates(subset=['route_id'])

In [239]:
#Route lines merged with name, routes.txt
shape_routes = shape_merge.merge(routes, left_on='route_id', right_on='route_id', how='inner')

In [240]:
shape_routes.crs = {'init' :'epsg:4326'}

/Applications/anaconda3/envs/mapping/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


## Preparing datasets to publish on a map

### Converting files into GeoJson

In [241]:
#stops, convert to 4326 in case it was in some other format
sjson = stops_gdf.to_json()

In [242]:
#routes converted to GeoJson
rjson = shape_routes.to_json()

In [243]:
shape_routes.head()

,shape_id,geometry,route_id,service_id,trip_id,trip_headsign,block_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,gm0,"LINESTRING (-77.27147 38.87847, -77.27140 38.8...",green2,all-f,31,To George Mason University,64,1,green2,Green 2,NaN,3,NaN,009933,ffffff
1,gm1,"LINESTRING (-77.27147 38.87847, -77.27140 38.8...",green1,all-f,171,To George Mason University,54,1,green1,Green 1,NaN,3,NaN,009933,ffffff
2,gm2,"LINESTRING (-77.33241 38.85606, -77.33271 38.8...",gold2,all-f,281,To George Mason University,83,1,gold2,Gold 2,NaN,3,NaN,ff9900,000000
3,gm4,"LINESTRING (-77.27147 38.87847, -77.27140 38.8...",gold1,all-mtwh,502,To George Mason University,73,1,gold1,Gold 1,NaN,3,NaN,ff9900,000000


## Mapping Options

In [244]:
# Function identifying centroid of routes to display original location of map
def get_feature_centroid(gdf):
    gdf['dissolve'] = 1
    center = gdf.dissolve(by='dissolve').centroid
    return center.x, center.y

In [245]:
# Getting Sentriod to display original coordinates for map.
x, y = get_feature_centroid(shape_routes)

# Setting up Basemap
m = folium.Map(
    location=[y, x],
    zoom_start=10,
    tiles='cartodbpositron') 

# Function to return route color with # in front to display properly for leaflet style_function
def style_geojson(features):
   # if features['properties']['route_color'] == True:
        return {'color': f"#{features['properties']['route_color']}"} 
    #else:
    #    return {'color': 'blue'}
    
# Display GeoJson of Routes
folium.GeoJson(
    rjson,
    name= 'Routes',
    style_function=style_geojson,
).add_to(m)

# Creating and displaying folium.Marker for every stop
for index,row in stops_gdf.iterrows():
    x, y = row['geometry'].centroid.x, row['geometry'].centroid.y
    folium.Circle(
        location = [y, x],
        radius=20,
        popup=" Station Name:" + row['stop_name'] + "Routes Served:",
        color='black',
        fill=True,
        fill_opacity=500,
    ).add_to(m)
#folium.LayerControl().add_to(m)
m

## Creating HTML File for Map

In [224]:
m.save(f"{html_name}.html")